# Bias wave stability

Looking at the impact of sequencer changes on the bias wave\
and large variations at the beginning of the serial read.\
Craig Lage - 07-Feb-23 - Sequencer files were created by Yousuke Utsumi and Stuart Marshall

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.ip.isr import IsrTask, IsrTaskConfig
import lsst.afw.display as afwDisplay
import lsst.geom as geom

In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])

## First, we look at the impact of the more conventional sequencer changes

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
plt.title("AuxTel bias stability - Amp C10")
names = ['Baseline', 'noTL_noRG', 'noTL_noS2', 'v26_noRG', 'Baseline']
for i, expId in enumerate([2023022200020, 2023022200040, 2023022200060, 2023022200080, 2023022200100]):
    exp = butler.get('raw', detector=0, exposure=expId)
    isrResult = isrTask.run(exp)
    arr = isrResult.exposure.image.array
    plt.plot(np.mean(arr[2100:3900, 0:400], axis=0), label=f"{expId}:{names[i]}")
    plt.ylim(-6,6)
plt.legend(bbox_to_anchor=(0.45, 0.4))
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
plt.savefig("/home/c/cslage/u/AuxTel/isr/sequencer_tests_22feb23/Bias_Stability_Sequencer_Amp_C10.pdf")

In [ ]:
bias = butler.get('bias', detector=0, exposure=2022110800013)
biasArr = bias.image.array
plt.title("AuxTel bias stability - Amp C10")
plt.subplot(2,1,1)
names = ['Baseline', 'noTL_noRG', 'noTL_noS2', 'v26_noRG', 'Baseline']
for i, expId in enumerate([2023022200020, 2023022200040, 2023022200060, 2023022200080, 2023022200100]):
    exp = butler.get('raw', detector=0, exposure=expId)
    #biasExp = butler.get('bias', detector=0, exposure=expId)
    isrResult = isrTask.run(exp)
    arr = isrResult.exposure.image.array
    plt.plot(np.mean(arr[2100:3900, :], axis=0), label=f"{expId}-{names[i]}")
plt.ylim(-60,60)
#plt.plot(np.mean(biasArr[2100:3900, :], axis=0), ls='--', lw=2, color='black', label="Master bias")
plt.legend(bbox_to_anchor=(0.8, -0.3))
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
plt.xlim(475,509)
plt.subplot(2,1,2)
plt.axis('off')
plt.savefig("/home/c/cslage/u/AuxTel/isr/sequencer_tests_22feb23/Bias_Stability_2_Sequencer_Amp_C10.pdf")

## Next, we look at an image with the parallel "underscan" 
### For this, we need to do the serial overscan subtraction manually
### These images, instead of the usual 48 parallel overscan rows, have 10 parallel underscan rows and 38 parallel overscan rows.

In [ ]:
# This is pulled out of ip_isr/overscan.py
# It basically takes a 1D array of overscan values and "broadcasts"
# then to 1 2D array that can be subtracted from the whole amp array
def broadcastFitToImage(overscanValue, imageArray, transpose=False):
        if isinstance(overscanValue, np.ndarray):
            overscanModel = np.zeros_like(imageArray)

            if transpose is False:
                if imageArray.shape[0] == overscanValue.shape[0]:
                    overscanModel[:, :] = overscanValue[:, np.newaxis]
                elif imageArray.shape[1] == overscanValue.shape[0]:
                    overscanModel[:, :] = overscanValue[np.newaxis, :]
                elif imageArray.shape[0] == overscanValue.shape[1]:
                    overscanModel[:, :] = overscanValue[np.newaxis, :]
                else:
                    raise RuntimeError(f"Could not broadcast {overscanValue.shape} to "
                                       f"match {imageArray.shape}")
            else:
                if imageArray.shape[1] == overscanValue.shape[0]:
                    overscanModel[:, :] = overscanValue[np.newaxis, :]
                elif imageArray.shape[0] == overscanValue.shape[0]:
                    overscanModel[:, :] = overscanValue[:, np.newaxis]
                elif imageArray.shape[1] == overscanValue.shape[1]:
                    overscanModel[:, :] = overscanValue[:, np.newaxis]
                else:
                    raise RuntimeError(f"Could not broadcast {overscanValue.shape} to "
                                       f"match {imageArray.shape}")
        else:
            overscanModel = overscanValue

        return overscanModel

### First, we run an image with parallel underscan

In [ ]:
# This subtracts the serial overscan region
skipCols = 4 # Number of columns to skip at the front and back of the serial overscan region
expCounter = 0
# Average 10 biases to help with the statistics
for expId in range(2023022100053, 2023022100071): # Using parallel underscan
    exposure = butler.get('raw', detector=0, exposure=expId)
    isTransposed = False
    for amp in exposure.getDetector().getAmplifiers():
        serialOverscanBBox = amp.getRawSerialOverscanBBox()
        imageBBox = amp.getRawDataBBox()
        parallelOverscanBBox = amp.getRawParallelOverscanBBox()
        imageBBox = imageBBox.expandedTo(parallelOverscanBBox)

        serialOverscanBBox = geom.Box2I(geom.Point2I(serialOverscanBBox.getMinX()+skipCols,
                                                     imageBBox.getMinY()),
                                        geom.Extent2I(serialOverscanBBox.getWidth()-2*skipCols,
                                                      imageBBox.getHeight()))
        overscan = np.nanmedian(exposure[serialOverscanBBox].image.array, axis=1)
        imageArray = exposure[amp.getRawBBox()].image.array
        overscanModel = broadcastFitToImage(overscan, imageArray, transpose=False)
        exposure[amp.getRawBBox()].image.array -= overscanModel
    if expCounter == 0:
        arr = exposure.image.array
    else:
        arr += exposure.image.array
    expCounter += 1
arr /= float(expCounter)

In [ ]:
ymin = 2200
ymax = 3900
arrMedian = np.mean(arr[ymin:ymax, :], axis=0)
fig = plt.figure(figsize=(8,12))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"With parallel underscan - {expId}", fontsize=18)
plt.subplot(3,2,1)
plt.title(f"Wave: Imaging region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(0,450)
plt.ylim(-6,6)
plt.subplot(3,2,2)
plt.title(f"Start of serial: Imaging region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(538, 572)
plt.ylim(-40,40)
ymin = 2050
ymax = 2085
arrMedian = np.mean(arr[ymin:ymax, :], axis=0)
plt.subplot(3,2,3)
plt.title(f"Wave: Parallel overscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(0,450)
plt.ylim(-6,6)
plt.subplot(3,2,4)
plt.title(f"Start of serial: Parallel overscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(538, 572)
plt.ylim(-40,40)
ymin = 4087
ymax = 4094
arrMedian = np.mean(arr[ymin:ymax, :], axis=0)
plt.subplot(3,2,5)
plt.title(f"Wave: Parallel underscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(0,450)
plt.ylim(-6,6)
plt.subplot(3,2,6)
plt.title(f"Start of serial: Parallel underscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(538, 572)
plt.ylim(-40,40)
plt.savefig("/home/c/cslage/u/AuxTel/isr/sequencer_tests_22feb23/Parallel_Underscan_UR10_NoTL_NoS2_Amp_C10.pdf")

In [ ]:
arr = exposure.image.array
ymin = 2200
ymax = 3900
arrMedian = np.mean(arr[ymin:ymax, :], axis=0)
fig = plt.figure(figsize=(8,12))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"With parallel underscan - {expId}", fontsize=18)
plt.subplot(3,2,1)
plt.title(f"Wave: Imaging region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(0+576,450+576)
plt.ylim(-6,10)
plt.subplot(3,2,2)
plt.title(f"Start of serial: Imaging region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(538+576, 572+576)
plt.ylim(-40,40)
ymin = 2050
ymax = 2085
arrMedian = np.mean(arr[ymin:ymax, :], axis=0)
plt.subplot(3,2,3)
plt.title(f"Wave: Parallel overscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(0+576,450+576)
plt.ylim(-6,10)
plt.subplot(3,2,4)
plt.title(f"Start of serial: Parallel overscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(538+576, 572+576)
plt.ylim(-40,40)
ymin = 4087
ymax = 4094
arrMedian = np.mean(arr[ymin:ymax, :], axis=0)
plt.subplot(3,2,5)
plt.title(f"Wave: Parallel underscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(0+576,450+576)
plt.ylim(-6,10)
plt.subplot(3,2,6)
plt.title(f"Start of serial: Parallel underscan region \n y={ymin}:{ymax}")
plt.plot(arrMedian)
plt.xlim(538+576, 572+576)
plt.ylim(-40,40)
plt.savefig("/home/c/cslage/u/AuxTel/isr/sequencer_tests_07feb23/Parallel_Underscan_Corrected_Amp_C11.pdf")

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.suptitle("AuxTel bias stability")
names = ['Baseline', 'noTL_noRG', 'noTL_noS2', 'v26_noRG', 'Baseline']
xplot = np.arange(509)
for i, expId in enumerate([2023022200020, 2023022200040, 2023022200060, 2023022200080, 2023022200100]):
    exp = butler.get('raw', detector=0, exposure=expId)
    isrResult = isrTask.run(exp)
    plotCounter = 1
    for amp in exp.getDetector().getAmplifiers():
        ampName = amp.getName()
        bbox = amp.getBBox()
        subExp = isrResult.exposure[bbox]
        arr = subExp.image.array
        plt.subplot(4,4,plotCounter)
        plt.title(ampName)
        plt.plot(xplot, np.mean(arr[100:1900, :], axis=0))
        plt.ylim(-20,20)
        if plotCounter % 4 == 1:
            plt.ylabel("Signal(ADU)")
        else:
            plt.yticks([])
        if plotCounter > 12:
            plt.xlabel("Xcoordinate(pixels)")
        else:
            plt.xticks([])
        plotCounter += 1

plt.savefig("/home/c/cslage/u/AuxTel/isr/sequencer_tests_22feb23/Bias_Stability_Sequencer_All_Amps.pdf")